# Analyze results of Timeloop/Accelergy on NeRF

In [1]:
import json
import os
import pandas as pd
import regex as re

In [2]:
!ls profile_results

eyeriss_like_onchip_compression_results.json
eyeriss_like_results.json
eyeriss_like_w_gating_results.json
results.json
simba_like_results.json
simple_weight_stationary_results.json


## Load Results from the Profile notebok

In [3]:
def load_results(results_path: str = "profile_results/results.json") -> dict:
    """ Load results from Profile NeRF.ipynb notebook back into original shape. """
    with open(results_path, "r") as f:
        results = json.load(f)
    print(f"Loaded results from {results_path}")

    for arch, arch_result_dict in results.items():
        arch_results_path = arch_result_dict["results"]
        assert os.path.exists(arch_results_path), f"Could not find {arch_results_path} for {arch}"

        with open(arch_results_path, "r") as f:
            arch_result_dict["results"] = json.load(f)    
        print(f"Populated {arch} results from {arch_results_path}")
        
    return results

profile_results = load_results()

Loaded results from profile_results/results.json
Populated eyeriss_like results from /home/workspace/notebooks/profile_results/eyeriss_like_results.json
Populated eyeriss_like_onchip_compression results from /home/workspace/notebooks/profile_results/eyeriss_like_onchip_compression_results.json
Populated eyeriss_like_w_gating results from /home/workspace/notebooks/profile_results/eyeriss_like_w_gating_results.json


## Analyze

In [4]:
# Compute the actual area as the summary got it wrong
area_regex = r"Area.*:\s(\d+.\d+)"

for arch, arch_results in profile_results.items():
    arch_total_area = 0.0
    layer_summary = arch_results["layer_summary"]
    
    for layer_id in layer_summary:
        # Read all areas from the timeloop stats file
        stats_fname = layer_summary[layer_id]["stats_fname"]
        with open(stats_fname, "r") as f:
            stats = [l.strip() for l in f.readlines()]

        area_lines = [line for line in stats[:-50] if re.match(area_regex, line)]
        areas = [re.search(area_regex, line).group(1) for line in area_lines]
        areas = [float(area) for area in areas]
        area = sum(areas)
        
        # Need to multiply by number of this layer type
        num = layer_summary[layer_id]["num"]
        total_area = area * num
        
        layer_summary[layer_id]["actual_total_area"] = total_area
        layer_summary[layer_id]["actual_area"] = area
        arch_total_area += total_area
        
    arch_results["summary"]["actual_total_area"] = arch_total_area

In [5]:
# Massage results into dataframes
layer_dfs = {}
all_summary = {}
# Remove area and total_area as they are wrong as just 0.0
drop_cols = ["name", "stats_fname", "total_area", "area"]

for arch, arch_results in profile_results.items():
    all_summary[arch] = arch_results["summary"]

    # Load layer results into dataframe
    df_layer = pd.DataFrame.from_dict(arch_results["layer_summary"], orient="index")
    df_layer = df_layer.drop(columns=drop_cols)
    df_layer.index.name = "layer_id"
    layer_dfs[arch] = df_layer
    
df_summary = pd.DataFrame.from_dict(all_summary, orient="index")
df_summary = df_summary.drop(columns=["total_area"])

In [6]:
layer_dfs["eyeriss_like"]

,total_energy,total_cycle,num,energy,cycle,gflops,utilization,edp,shape,actual_total_area,actual_area
layer_id,,,,,,,,,,,
1,13.82,10752,1,13.82,10752,190.48,0.57,0.14900,"{'C': 63, 'M': 256, 'N': 64}",1375449.72,1375449.72
2,260.40,458752,7,37.20,65536,127.75,0.38,2.44000,"{'C': 256, 'M': 256, 'N': 64}",9628148.04,1375449.72
6,59.17,59392,1,59.17,59392,175.72,0.52,3.51000,"{'C': 319, 'M': 256, 'N': 64}",1375449.72,1375449.72
9,2.84,2048,1,2.84,2048,15.97,0.05,0.00581,"{'C': 256, 'M': 1, 'N': 64}",1375449.72,1375449.72
11,315.54,36224,1,315.54,36224,127.77,0.38,11.40000,"{'C': 283, 'M': 128, 'N': 64}",1375449.72,1375449.72
12,1.56,1024,1,1.56,1024,47.81,0.14,0.00160,"{'C': 128, 'M': 3, 'N': 64}",1375449.72,1375449.72


In [7]:
layer_dfs["eyeriss_like_onchip_compression"]

,total_energy,total_cycle,num,energy,cycle,gflops,utilization,edp,shape,actual_total_area,actual_area
layer_id,,,,,,,,,,,
1,12.91,10752,1,12.91,10752,190.48,0.57,0.13900,"{'C': 63, 'M': 256, 'N': 64}",907149.12,907149.12
2,41.10,65536,1,41.10,65536,127.75,0.38,2.69000,"{'C': 256, 'M': 256, 'N': 64}",907149.12,907149.12
3,41.10,65536,1,41.10,65536,127.75,0.38,2.69000,"{'C': 256, 'M': 256, 'N': 64}",907149.12,907149.12
4,41.10,65536,1,41.10,65536,127.75,0.38,2.69000,"{'C': 256, 'M': 256, 'N': 64}",907149.12,907149.12
5,41.10,65536,1,41.10,65536,127.75,0.38,2.69000,"{'C': 256, 'M': 256, 'N': 64}",907149.12,907149.12
6,58.37,59392,1,58.37,59392,175.72,0.52,3.47000,"{'C': 319, 'M': 256, 'N': 64}",907149.12,907149.12
7,41.10,65536,1,41.10,65536,127.75,0.38,2.69000,"{'C': 256, 'M': 256, 'N': 64}",907149.12,907149.12
8,41.10,65536,1,41.10,65536,127.75,0.38,2.69000,"{'C': 256, 'M': 256, 'N': 64}",907149.12,907149.12
9,2.58,2048,1,2.58,2048,15.97,0.05,0.00529,"{'C': 256, 'M': 1, 'N': 64}",907149.12,907149.12


In [8]:
layer_dfs["eyeriss_like_w_gating"]

,total_energy,total_cycle,num,energy,cycle,gflops,utilization,edp,shape,actual_total_area,actual_area
layer_id,,,,,,,,,,,
1,11.26,10752,1,11.26,10752,190.48,0.57,0.12100,"{'C': 63, 'M': 256, 'N': 64}",854095.4,854095.4
2,35.13,65536,1,35.13,65536,127.75,0.38,2.30000,"{'C': 256, 'M': 256, 'N': 64}",854095.4,854095.4
3,35.13,65536,1,35.13,65536,127.75,0.38,2.30000,"{'C': 256, 'M': 256, 'N': 64}",854095.4,854095.4
4,35.13,65536,1,35.13,65536,127.75,0.38,2.30000,"{'C': 256, 'M': 256, 'N': 64}",854095.4,854095.4
5,35.13,65536,1,35.13,65536,127.75,0.38,2.30000,"{'C': 256, 'M': 256, 'N': 64}",854095.4,854095.4
6,46.94,59392,1,46.94,59392,175.72,0.52,2.79000,"{'C': 319, 'M': 256, 'N': 64}",854095.4,854095.4
7,35.13,65536,1,35.13,65536,127.75,0.38,2.30000,"{'C': 256, 'M': 256, 'N': 64}",854095.4,854095.4
8,35.13,65536,1,35.13,65536,127.75,0.38,2.30000,"{'C': 256, 'M': 256, 'N': 64}",854095.4,854095.4
9,2.56,2048,1,2.56,2048,15.97,0.05,0.00524,"{'C': 256, 'M': 1, 'N': 64}",854095.4,854095.4


In [9]:
df_summary

,total_energy,total_cycle,num_params,macs,activation_size,actual_total_area
eyeriss_like,653.33,568192.0,595844,593450,2300.0,16505396.64
eyeriss_like_onchip_compression,416.01,568192.0,595844,593450,2300.0,10885789.44
eyeriss_like_w_gating,358.01,568192.0,595844,593450,2300.0,10249144.80
